# SimJulia tutorial examples

See https://benlauwens.github.io/SimJulia.jl/stable/tutorial.html

Set up local project environment with `] instantiate` (looks for .toml files in the same directory), and import packages. This will install SimJulia and PyPlot if not already installed.

In [1]:
] instantiate

In [2]:
using ResumableFunctions
using SimJulia

## Simple simulation

Set up car process. The `@resumable` decorator makes this a resumable function which will run continuously, but pass control flow back to the simulation once a `@yield` statement is reached.

In [3]:
@resumable function car(env::Environment)
           while true
             println("Start parking at ", now(env))
             parking_duration = 5
             @yield timeout(env, parking_duration)
             println("Start driving at ", now(env))
             trip_duration = 2
             @yield timeout(env, trip_duration)
           end
         end

car (generic function with 1 method)

In [4]:
# Set up simulation environment
sim = Simulation()
# Set up starting process (pass environment to process)
@process car(sim)
# Run simulation (pass simulation time)
run(sim, 15)

Start parking at 0.0
Start driving at 5.0
Start parking at 7.0
Start driving at 12.0
Start parking at 14.0


We could continue running the simulation to a later simulation time:

In [5]:
run(sim, 30)

Start driving at 19.0
Start parking at 21.0
Start driving at 26.0
Start parking at 28.0


## Waiting for a process

Now the car is going to be charged, another process, which takes a given time.

In [6]:
# Define charging process
@resumable function charge(env::Environment, duration::Number)
         @yield timeout(env, duration)
end

# Car process
@resumable function car(env::Environment)
         while true
           # Start charging 
           println("Start parking and charging at ", now(env))
           charge_duration = 5
           charge_process = @process charge(sim, charge_duration)
           @yield charge_process
           # Charging has ended. Drive.
           println("Start driving at ", now(env))
           trip_duration = 2
           @yield timeout(sim, trip_duration)
         end
       end

# Set up simulation environment
sim = Simulation()
# Set up starting process (pass environment to process)
@process car(sim)
# Run simulation (pass simulation time)
run(sim, 15)

Start parking and charging at 0.0
Start driving at 5.0
Start parking and charging at 7.0
Start driving at 12.0
Start parking and charging at 14.0


## Interrupting a process

Imagine we want to interupt charging, and start driving. We can use the `interrupt` function.

The `try` and `catch` syntax will handle what to do if a process is interrupted.

In [7]:
# After waiting three time steps the driver function interupts the car process
@resumable function driver(env::Environment, car_process::Process)
         @yield timeout(env, 3)
         @yield interrupt(car_process)
       end

# Define charging process
@resumable function charge(env::Environment, duration::Number)
         @yield timeout(env, duration)
       end

# Derfine car process
@resumable function car(env::Environment)
         while true
           println("Start parking and charging at ", now(env))
           charge_duration = 5
           charge_process = @process charge(sim, charge_duration)
           try
             @yield charge_process
           catch
             println("Was interrupted. Hopefully, the battery is full enough ...")
           end
           println("Start driving at ", now(env))
           trip_duration = 2
           @yield timeout(sim, trip_duration)
         end
       end

# Set up simulation environment
sim = Simulation()
# Set up starting processes (pass environment to process)
car_process = @process car(sim)
@process driver(sim, car_process)
# Run simulation (pass simulation time)
run(sim, 15)

Start parking and charging at 0.0
Was interrupted. Hopefully, the battery is full enough ...
Start driving at 3.0
Start parking and charging at 5.0
Start driving at 10.0
Start parking and charging at 12.0


## Shared resources

Resources are handled using SimJulia's `Resource` class. We will set up a battery charging station (BCS) resource. We will request and release resources with `@request` and `@release`.

In this example we will not run the simulation with a continuous `while` loop. Instead we will pass four process calls to the simulation -  the simulation will automatically stop when there are no more events left.

In [11]:
@resumable function car(env::Environment, name::Int, 
        bcs::Resource, driving_time::Number, charge_duration::Number)
         @yield timeout(sim, driving_time)
         println(name, " arriving at ", now(env))
         # Reuqest resource
         @yield request(bcs, priority=2)
         println(name, " starting to charge at ", now(env))
         @yield timeout(sim, charge_duration)
         println(name, " leaving the bcs at ", now(env))
         # Resources must be released
         @yield release(bcs)
       end

# Set up simulation environment
sim = Simulation()
# Set up resource
bcs = Resource(sim, 2)

# Simulate four cars
for i in 1:4
         @process car(sim, i, bcs, 2i, 5)
       end

run(sim)

1 arriving at 2.0
1 starting to charge at 2.0
2 arriving at 4.0
2 starting to charge at 4.0
3 arriving at 6.0
1 leaving the bcs at 7.0
3 starting to charge at 7.0
4 arriving at 8.0
2 leaving the bcs at 9.0
4 starting to charge at 9.0
3 leaving the bcs at 12.0
4 leaving the bcs at 14.0


In [15]:
function mysum(a, b=0)
    a + b
end

mysum(2, 1)

3